<a href="https://colab.research.google.com/github/kthom93/InternationalCinemaApp/blob/master/Secret_Hitler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 470 Lab 5: Modeling Secret Hitler
### Kevin Thompson and Emma Hunt

Secret Hitler is a partial information game where a group of players start off not knowing anything about each other. The players perceptions of others and their roles change throughout the game depending on the actions each agent takes.

#### From the Secret Hitler Game Maual

### Overview

At the beginning of the game, each player
is secretly assigned to one of three roles: Liberal, Fascist, or Hitler. The Liberals have a majority, but they don’t know for sure who anyone is; Fascists must resort to secrecy and sabotage to accomplish their goals. Hitler plays for the Fascist team, and the Fascists know Hitler’s identity from the outset, but Hitler doesn’t know the Fascists and must work to figure them out.

The Liberals win by enacting five Liberal Policies or killing Hitler. The Fascists win by enacting six Fascist Policies, or if Hitler is elected Chancellor after three Fascist Policies have been enacted.

Whenever a Fascist Policy is enacted, the government becomes more powerful, and the President is granted a single-use power which must be used before the next round can begin. It doesn’t matter what team the President is on; in fact, even Liberal players might be tempted to enact a Fascist Policy to gain new powers. 

### Object

Every player has a secret identity as a member of either the liberal team or the Fascist team.

Players on the Liberal team win if either:

  * Five Liberal Policies are enacted

  OR
  * Hitler is assassinated.

Players on the Fascist team win if either:

  * Six Fascist Policies are enacted.

  OR
  * Hitler is elected Chancellor any time after the third Fascist Policy has been enacted.

## Game Setup

In [0]:
import random

from enum import Enum

# So we don't have to look at index numbers all day
names = ['James', 'Mary', 'John', 'Patricia', 'Robert', 'Jennifer', 'Michael', 'Linda', 'William', 'Elizabeth']

# total number of each policy in the deck
TOT_FAS_POL = 11
TOT_LIB_POL = 6

# A player can either be Liberal, Fascist or Hitler
class Role(Enum):
  LIBERAL = 1
  FASCIST = 2
  HITLER = 3

# Policies of the game can either be Liberal or Fascist
class Policy(Enum):
  LIBERAL = 0
  FASCIST = 1

# The belief from one player to another for each role.
# 1.0 means total belief in there role
# 0.0 means total disbelief in there role
# Fascist players only have absolute knowledge
# TODO: consider adding partial history object that allows historical actions to influence numbers
class Belief():
  def __init__(self, lib, fas, hit):
    self.bel_lib = lib
    self.bel_fas = fas
    self.bel_hit = hit
  # Reporting
  def get_info(self):
    output = ''
    output += 'Belief Liberal: ' + str(self.bel_lib) + '\n'
    output += 'Belief Fascist: ' + str(self.bel_fas) + '\n'
    output += 'Belief Hitler: ' + str(self.bel_hit) + '\n'
    return output

# The player agent fo the game
class Player():
  def __init__(self, role, name):
    self.name = name
    self.role = role
    # Dictionary from other player to belief object
    self.beliefs = {}

  def update_beliefs(self, history, fas_policy_count, lib_policy_count):
    if self.role == Role.FASCIST or history[-1].passed == False or history[-1].killed == self:
      return # Already have perfect information
    last_chancellor = history[-1].chancellor
    last_president = history[-1].president
    if self == last_president and history[-1].killed != last_chancellor:
      # I am president, so I know what cards chancellor got
      # If I gave them a choice, their choice suggests they are that party
      lib_count = 0
      fas_count = 0
      for card in history[-1].chan_hand:
        if card == Policy.FASCIST:
          fas_count += 1
        else:
          lib_count += 1
      if fas_count == 1 and lib_count == 1:
        # I gave them a choice
        if history[-1].policy == Policy.FASCIST:
          # Chose a Fascist policy, so they are Fascist, and maybe Hitler
          self.beliefs[last_chancellor].bel_fas = 1.0
          self.beliefs[last_chancellor].bel_lib = 0.0
          if self.role != Role.HITLER:
            self.beliefs[last_chancellor].bel_hit = min(1.0, max(0.25, self.beliefs[last_chancellor].bel_lib * 1.25))
          # If they played Fascist there is a 1/4 chance they are Hitler, so increase belief by 25%
        else:
          # Chose a Liberal policy, so Liberal or Hitler playing Liberal
          self.beliefs[last_chancellor].bel_lib = min(1.0, self.beliefs[last_chancellor].bel_lib * 1.9)
          self.beliefs[last_chancellor].bel_fas = max((1.0/7.0), self.beliefs[last_chancellor].bel_fas * 0.8)
          if self.role != Role.HITLER:
            self.beliefs[last_chancellor].bel_hit = self.beliefs[last_chancellor].bel_hit * (8.0/7.0)
            # If they played Liberal, there is still a 1/7 chance they are Hitler
      if fas_policy_count > 3:
        self.beliefs[last_chancellor].bel_hit = 0.0
      return # President is now up to date
    elif self == last_chancellor:
      # I am Chancellor, i know what cards the president gave me
      # If President gave me a Liberal card, they are probably Liberal
      lib_count = 0
      fas_count = 0
      liberal_in_deck = TOT_LIB_POL - lib_policy_count
      fascist_in_deck = TOT_FAS_POL - fas_policy_count
      number_in_deck = (TOT_LIB_POL + TOT_FAS_POL) - (fas_policy_count + lib_policy_count)
      odds_2_liberal = (liberal_in_deck / number_in_deck) * ((liberal_in_deck-1) / (number_in_deck-1))
      odds_3_liberal = odds_2_liberal * ((liberal_in_deck-2)/(number_in_deck-2))
      odds_all_fascist = (fascist_in_deck / number_in_deck) * ((fascist_in_deck-1) / (number_in_deck-1)) * ((fascist_in_deck-2) / (fascist_in_deck-2))
      for card in history[-1].chan_hand:
        if card == Policy.FASCIST:
          fas_count += 1
        else:
          lib_count += 1
      if lib_count == 1:
        # odds_2_liberal they are Fascist still
        self.beliefs[last_president].bel_lib = min(1.0, max(1-odds_2_liberal, self.beliefs[last_president].bel_lib * (2-odds_2_liberal)))
        self.beliefs[last_president].bel_fas = min(odds_2_liberal, self.beliefs[last_president].bel_fas * odds_2_liberal)
        if self.role != Role.HITLER:
          self.beliefs[last_president].bel_hit = self.beliefs[last_president].bel_hit * 0.8
      elif lib_count == 2:
        # odds_3_liberal they are Fascist still
        self.beliefs[last_president].bel_lib = min(1.0, max(1-odds_3_liberal, self.beliefs[last_president].bel_lib * (2-odds_3_liberal)))
        self.beliefs[last_president].bel_fas = max(odds_3_liberal, self.beliefs[last_president].bel_fas * odds_3_liberal)
        if self.role != Role.HITLER:
          self.beliefs[last_president].bel_hit = self.beliefs[last_president].bel_hit * 0.8
      else:
        # got all Fascist cards
        # still odds_all_fascist they are liberal
        self.beliefs[last_president].bel_lib = max(odds_all_fascist, self.beliefs[last_president].bel_lib * odds_all_fascist)
        self.beliefs[last_president].bel_fas = min(1.0, max(1-odds_all_fascist, self.beliefs[last_president].bel_fas * (2-odds_all_fascist)))
        if self.role != Role.HITLER:
          self.beliefs[last_president].bel_hit = self.beliefs[last_president].bel_hit * 1.2
    else:
      # Not President or Chancellor
      # Update President
      if history[-1].policy == Policy.LIBERAL:
        # Liberal policy passed, so both pres and chan are likely liberal
        # president up lib, down fas, down hit
        self.beliefs[last_president].bel_lib = min(1.0, self.beliefs[last_president].bel_lib * 1.4)
        self.beliefs[last_president].bel_fas = self.beliefs[last_president].bel_fas * 0.7
        if self.role != Role.HITLER:
          self.beliefs[last_president].bel_hit = self.beliefs[last_president].bel_hit * (6.0/7.0) # decrease by 1/7
      else:
        # Fascist policy passed, so slightly more likely both are Fascist
        self.beliefs[last_president].bel_lib = self.beliefs[last_president].bel_lib * 0.7
        self.beliefs[last_president].bel_fas = min(1.0, self.beliefs[last_president].bel_fas * 1.3)
        if self.role != Role.HITLER:
          self.beliefs[last_president].bel_hit = min(1.0, self.beliefs[last_president].bel_hit * 1.3)
      if history[-1].killed != last_chancellor:
        # Chancellor is still alive
        if history[-1].policy == Policy.LIBERAL:
          # Liberal policy passed, so both pres and chan are likely liberal, though chan could be hitler
          # Chancellor up lib, down fas, up hit
          self.beliefs[last_chancellor].bel_lib = min(1.0, self.beliefs[last_chancellor].bel_lib * 1.4)
          self.beliefs[last_chancellor].bel_fas = self.beliefs[last_chancellor].bel_fas * 0.6
          if self.role != Role.HITLER:
            self.beliefs[last_chancellor].bel_hit = self.beliefs[last_chancellor].bel_hit * (6.0/7.0)
        else:
          # fascist policy passed, so slightly more likely both are fascist
          self.beliefs[last_chancellor].bel_lib = self.beliefs[last_chancellor].bel_lib * 0.7
          self.beliefs[last_chancellor].bel_fas = min(1.0, self.beliefs[last_chancellor].bel_fas * 1.3)
          if self.role != Role.HITLER:
            self.beliefs[last_chancellor].bel_hit = min(1.0, self.beliefs[last_chancellor].bel_hit * 1.2)
        if fas_policy_count > 3:
          # we know this chancellor is not hitler, otherwise the game would be over
          self.beliefs[last_chancellor].bel_hit = 0.0
    if history[-1].reveal != None and self != last_president:
      # need to update person who was revealed based on info
      person = history[-1].reveal[0]
      party = history[-1].reveal[1]
      if person == self:
        # if the president lied about you, you know they are a fascist
        if party != self.role:
          self.beliefs[last_president].bel_fas = 1.0
          self.beliefs[last_president].bel_lib = 0.0
          if self.role != Role.HITLER:
            self.beliefs[last_president].bel_hit = max(0.25, self.beliefs[last_president].bel_hit * (5.0/4.0))
        else:
          # not a fascist, either liberal or hitler
          self.beliefs[last_president].bel_fas = 0.0
          self.beliefs[last_president].bel_lib = self.beliefs[last_president].bel_lib * 1.5
          if self.role != Role.HITLER:
            self.beliefs[last_president].bel_hit = max((1.0/7.0), self.beliefs[last_president].bel_hit * (8.0/7.0))
      else:
        # your belief in the president telling the truth is your belief in their information
        pres_conf = self.beliefs[last_president].bel_lib - self.beliefs[last_president].bel_fas
        if pres_conf > 0:
          # you believe them
          if party == Role.LIBERAL:
            self.beliefs[person].bel_lib = min(1.0, self.beliefs[person].bel_lib + self.beliefs[person].bel_lib * pres_conf)
            self.beliefs[person].bel_fas = self.beliefs[person].bel_fas * (1-pres_conf)
            if self.role != Role.HITLER:
              self.beliefs[person].bel_hit = self.beliefs[person].bel_hit * (1-pres_conf)
          else:
            self.beliefs[person].bel_fas = min(1.0, self.beliefs[person].bel_fas + self.beliefs[person].bel_fas * pres_conf)
            self.beliefs[person].bel_lib = self.beliefs[person].bel_lib * (1-pres_conf)
            if self.role != Role.HITLER:
              self.beliefs[person].bel_hit = min(1.0, self.beliefs[person].bel_hit + self.beliefs[person].bel_hit * pres_conf)
        else:
          # you don't, do opposite
          if self.role != Role.HITLER:
            # you are hitler, you think the person who is president is fascist and lying
            if party == Role.LIBERAL:
              # you think they are fascist
              self.beliefs[person].bel_fas = min(1.0, self.beliefs[person].bel_fas + self.beliefs[person].bel_fas * -pres_conf)
              self.beliefs[person].bel_lib = self.beliefs[person].bel_lib * (1+pres_conf)
            else:
              # you think they are liberal
              self.beliefs[person].bel_lib = min(1.0, self.beliefs[person].bel_lib + self.beliefs[person].bel_lib * -pres_conf)
              self.beliefs[person].bel_fas = self.beliefs[person].bel_fas * (1+pres_conf)
          elif party == Role.LIBERAL:
            # could be a lie, could be said by hitler who always says liberal
            # figure out if you think they are hitler
            top_score = 0.0
            top_hitler = None
            for person, bel in self.beliefs.items():
              if bel.bel_hit > top_score:
                top_score = bel.bel_hit
                top_hitler = person
            if person == top_hitler:
              # 6/10 chance what he siad was true, slightly increas likelyhood they are liberal
              self.beliefs[person].bel_lib = min(1.0, self.beliefs[person].bel_lib * 1.2)
              self.beliefs[person].bel_fas = self.beliefs[person].bel_fas * 0.85
              self.beliefs[person].bel_hit = self.beliefs[person].bel_hit * 0.85
            else:
              # they said liberal, you think they are fascist
              self.beliefs[person].bel_fas = min(1.0, self.beliefs[person].bel_fas + self.beliefs[person].bel_fas * -pres_conf)
              self.beliefs[person].bel_lib = self.beliefs[person].bel_lib * (1 + pres_conf)
              self.beliefs[person].bel_hit = min(1.0, self.beliefs[person].bel_hit + self.beliefs[person].bel_hit * -pres_conf)
          else:
            # they said fascist, you think they are liberal
            # pres_conf is now negaitve
            self.beliefs[person].bel_lib = min(1.0, self.beliefs[person].bel_lib + self.beliefs[person].bel_lib * -pres_conf)
            self.beliefs[person].bel_fas = self.beliefs[person].bel_fas * (1 + pres_conf)
            self.beliefs[person].bel_hit = self.beliefs[person].bel_hit * (1 + pres_conf)


  # president updates belief about person updated
  def presidential_investigative_update(self, person, party):
    if party == Role.FASCIST:
      self.beliefs[person].bel_fas = 1.0
      self.beliefs[person].bel_lib = 0.0
      self.beliefs[person].bel_hit = max(self.beliefs[person].bel_hit * (5.0/4.0), 0.25)
    else: # party is liberal
      self.beliefs[person].bel_fas = 0.0
      self.beliefs[person].bel_lib = 1.0
      self.beliefs[person].bel_hit = 0.0
    
  
  # Look at history object, belief and party to make good nominations
  # If Fascist and if 3 or more Fascist policies have been passed, nominate Hitler
  # If Liberal and if 3 or more Fascist policies have been passed, do everything to not nominate Hitler
  def pick_chancellor(self, history, fas_policy_count, lib_policy_count, failed_nom_count):
    # Can't nominate someone who was President or Chancellor the last round
    last_chan = None
    last_pres = None

    if len(history) > 0:
      last_chan = history[-1].chancellor
      last_pres = history[-1].president

    # End game: Game ends if Hitler is elected Chancellor
    if fas_policy_count >= 3: 
      # Pick Hitler
      if self.role == Role.FASCIST:
        if last_chan.role != Role.HITLER and last_pres.role != Role.HITLER:
          for player, belief in self.beliefs.items():
            if belief.bel_hit == 1.0:
              return player
      # Don't pick Hitler based on beliefs
      # Hitler belief waited against Liberal and Fascist beliefs
      elif self.role == Role.LIBERAL:
        best_bel = 100.0
        best_player = None
        liberals = []
        for player, belief in self.beliefs.items():
          if player != last_chan and player != last_pres:
            if belief.bel_lib > belief.bel_fas:
              liberals.append(player)
        for player in liberals:
          if self.beliefs[player].bel_hit < best_bel:
            best_bel = self.beliefs[player].bel_hit
            best_player = player
        if best_player == None:
          worst_score = 100.0
          for player, belief in self.beliefs.items():
            if belief.bel_hit < worst_score:
              worst_score = belief.bel_hit
              best_player = player
        return best_player
      else:
        # Hitler: Pick a Fascist
        best_bel = -10.0
        best_player = None
        for player, belief in self.beliefs.items():
          if player != last_chan and player != last_pres:
            temp_bel = belief.bel_fas - belief.bel_lib
            if temp_bel > best_bel:
              best_bel = temp_bel
              best_player = player
        return best_player
  
    # Not the end game
    # Pick Fascist
    if self.role == Role.FASCIST:
      fas_list = []
      for player, belief in self.beliefs.items():
        if belief.bel_fas == 1.0 and player != last_pres and player != last_chan:
          fas_list.append(player)
      if len(fas_list) > 0 :
        return fas_list[random.randint(0, len(fas_list)-1)]
      else:
        # No avalible Fascists: Pick randomly
        nom = last_chan
        while nom == last_chan or nom == last_pres:
          nom = list(self.beliefs.keys())[random.randint(0, len(self.beliefs) - 1)]
        return nom
    elif self.role == Role.LIBERAL:
      best_bel = -100.0
      best_player = None
      for player, belief in self.beliefs.items():
        if player != last_chan and player != last_pres:
          temp_bel = belief.bel_lib - belief.bel_fas
          if temp_bel > best_bel:
            best_bel = temp_bel
            best_player = player
      return best_player
    else:
      # Hitler: Pick a Fascist
      best_bel = -100.0
      best_player = None
      for player, belief in self.beliefs.items():
        if player != last_chan and player != last_pres:
          temp_bel = belief.bel_fas - belief.bel_lib
          if temp_bel > best_bel:
            best_bel = temp_bel
            best_player = player
      return best_player
          
  # Voting for Chancellor nomination
  # If player is President or Chancellor, should vote yes
  # Player is Liberal and player believes that either President or
  # Chancellor is Fascist, should vote false
  # Player is Fascist and either President or Chancellor is Fascist,
  # should vote true
  def vote(self, president, chancellor, fas_policy_count): 
    if self == president or self == chancellor:
      return True
    if self.role == Role.FASCIST:
      if self.beliefs[president].bel_lib == 1.0 and self.beliefs[chancellor].bel_lib == 1.0:
        return False
      return True
    elif self.role == Role.LIBERAL:
      if fas_policy_count >= 3:
        if self.beliefs[chancellor].bel_hit > .15:
          return False
        if self.beliefs[president].bel_fas >=0.5:
          # the president probably nominated hitler
          self.beliefs[chancellor].bel_hit = min(1.0, self.beliefs[chancellor].bel_hit * 2)
          return False
      if self.beliefs[president].bel_fas >= 0.5 or self.beliefs[chancellor].bel_fas >= 0.5:
        return False
      return True
    else:
      # Hitler: Plays dumb
      return True

  # Returns policies to pass to Chancellor
  # TODO: could make this smarter: strictly party aligned currently
  def select_policies(self, hand):
    print('-----President Selection-----')
    if self.role == Role.FASCIST:
      for card in hand:
        if card == Policy.LIBERAL:
          hand.remove(card)
          break
    if self.role == Role.LIBERAL:
      for card in hand:
        if card == Policy.FASCIST:
          hand.remove(card)
          break
    # Treating Hitler as Fascist at the moment
    # TODO: Might change based on strategy, if he wants to appear liberal
    if self.role == Role.HITLER:
      for card in hand:
        if card == Policy.LIBERAL:
          hand.remove(card)
          break
    if len(hand) >= 3:
      hand.remove(hand[0])
    for card in hand:
      if card == Policy.FASCIST:
        print('Fascist Policy')
      else:
        print('Liberal Policy')
    print()
    return hand

  # Returns what policy gets played
  # Only called on a elected Chancellor
  # TODO: Might make Hitlers decisions more liberal: currently plays fascist
  # All players play strictly their party
  def play_policy(self, hand):
    print('-----Chancellor Selection-----')
    play = hand[0]
    for card in hand:
      if self.role == Role.LIBERAL and card == Policy.LIBERAL:
        play = card
        break
      if self.role == Role.FASCIST and card == Policy.FASCIST:
        play = card
        break
      if self.role == Role.HITLER and card == Policy.FASCIST:
        play =  card
        break
    if play == Policy.FASCIST:
      print('Fascist Policy')
    else:
      print('Liberal Policy')
    print()
    return play

  # Allows President to investigate another player
  # Returns who got investigated and what the President stated
  # Update personal beliefs at the moment from information
  #   Only applicable if Liberal/Hitler
  def investigate(self):
    # Random Selection
    person = list(self.beliefs.keys())[random.randint(0, len(self.beliefs) - 1)]
    # What the President sees
    knowledge = person.role
    if self.role != Role.FASCIST:
      self.presidential_investigative_update(person, knowledge)
    # What the President is going to say
    # Liberal tells truth, Fascist say opposite, hitler always says liberal
    if self.role == Role.LIBERAL:
      statement = knowledge
    elif self.role == Role.HITLER:
      statement = Role.LIBERAL
    else: # fascist, always lie
      if knowledge == Role.LIBERAL:
        statement = Role.FASCIST
      else:
        statement = Role.LIBERAL

    print('-----Investigation-----')
    print('Investigating: ' + person.name)
    if person.role == Role.LIBERAL:
      print('Role: Liberal')
    else:
      print('Role: Fascist')
    if statement == Role.LIBERAL:
      print('Statement: Liberal')
    else:
      print('Statement: Fascist')
    print()

    return person, statement

  # Chooses a player to assassinate
  # If Fascist, kill a liberal
  # If Liberal, try to kill Hitler
  # If Hitler, try to kill liberal
  def assassinate(self):
    # Random selection
    target = None
    if self.role == Role.LIBERAL:
      # liberal, try and kill hitler
      top_hitler = None
      top_score = 0.0
      for person, belief in self.beliefs.items():
        if belief.bel_hit > top_score:
          top_score = belief.bel_hit
          top_hitler = person
      target = top_hitler
    elif self.role == Role.HITLER:
      # hitler, try and kill least fascist
      worst_fas = None
      worst_score = 100.0
      for person, belief in self.beliefs.items():
        if belief.bel_fas < worst_score:
          worst_score = belief.bel_fas
          worst_fas = person
      target = worst_fas
    else:
      # fascist, kill a random liberal
      libs = []
      for person, belief in self.beliefs.items():
        if belief.bel_lib == 1.0:
          libs.append(person)
      target = libs[random.randint(0, len(libs)-1)]
    print('-----Assassination-----')
    print('Target: ' + target.get_info())
    print()
    return target

  # Remove assassinated person from beliefs
  def remove_from_beliefs(self, person):
    del self.beliefs[person]
    # TODO: Update other beliefs

  # Reporting
  # Returns Name and Party
  def get_info(self):
    if self.role == Role.FASCIST:
      return self.name + '-Fascist'
    if self.role == Role.LIBERAL:
      return self.name + '-Liberal'
    return self.name + '-Hitler' 


# Historical object for each agent's memory
class Turn():
  def __init__(self, president, chancellor, passed, policy=None, pres_hand=None, chan_hand=None, reveal=None, election=None, killed=None):
    self.president = president
    self.chancellor = chancellor
    # Whether the candidacy passed
    self.passed = passed
    # The policy that was enacted
    self.policy = policy
    # policies president recieved
    self.pres_hand = pres_hand
    # policies chacelor recieved
    self.chan_hand = chan_hand
    # Tuple of who was selected and what the President said
    self.reveal = reveal
    # Who the new President became
    self.election = election
    # The player killed
    self.killed = killed

In [0]:
# Helper Functions

# Initializes policy deck
# 11 Fascist policies
# 6 Liberal policies
def initialize_policies():
  deck = []
  for i in range(TOT_FAS_POL):
    deck.append(Policy.FASCIST)
  for i in range(TOT_LIB_POL):
    deck.append(Policy.LIBERAL)
  random.shuffle(deck)
  return deck

# Takes discarded policies and adds to bottom of policy deck
# Returns policy deck and discard pile
def reset_policies(policies, discard):
  random.shuffle(discard)
  policies += discard
  return policies, []

# Initializes role deck
# 6 Liberal roles
# 3 Fascist roles
# 1 Hitler role
def initialize_roles(num_players):
  roles = []
  # For now assume 10
  if num_players == 10:
    for i in range(6):
      roles.append(Role.LIBERAL)
    for i in range(3):
      roles.append(Role.FASCIST)
    roles.append(Role.HITLER)
  random.shuffle(roles)
  return roles

# Returns 3 policies
# Resets policy deck if less than 3 cards left
# Called when nomination passed and used by President
def get_policies(policies, discard):
  if len(policies) < 3:
    policies, discard = reset_policies(policies, discard)
  hand = policies[0:3]
  policies = policies[3:]
  discard += hand
  print('-----Getting Policies-----')
  for card in hand:
    if card == Policy.FASCIST:
      print('Fascist Policy')
    else:
      print('Liberal Policy')
  print()
  return hand, policies, discard

# Returns single policy
# Resets policy deck if less then 1 card left
# Called when 3 consecutive failed nominations happens
def get_next_policy(policies, discard):
  if len(policies) < 1:
    policies, discard = reset_policies(policies, discard)
  card = policies[0]
  policies = policies[1:]
  discard += [card]
  print('-----Failed Policy Selection-----')
  if card == Policy.FASCIST:
    print('Fascist Policy')
  else:
    print('Liberal Policy')
  print()
  return card, policies, discard

# Initializes players and there beliefs
def initialize_players(roles):
  players = []
  for i in range(len(roles)):
    players.append(Player(roles[i], names[i]))
  # Set beliefs
  for player in players:
    for other in players:
      if player != other:
        lib = 0.0
        fas = 0.0
        hit = 0.0
        # Fascist have perfect knowledge
        if player.role == Role.FASCIST:
          if other.role == Role.FASCIST:
            fas = 1.0
          elif other.role == Role.LIBERAL:
            lib = 1.0
          else:
            hit = 1.0
        # Liberal and Hitler have imperfect beliefs
        # Beliefs based on known game ratios
        elif player.role == Role.LIBERAL:
          lib = 5 / 9
          fas = 3 / 9
          hit = 1 / 9
        # Hitler
        else:
          lib = 6 / 9
          fas = 3 / 9
          hit = 0.0
        belief = Belief(lib, fas, hit)
        player.beliefs[other] = belief
  return players

# Checks the win status of the game
# True:
#   5 Liberal policies passed
#   6 Fascist policies passed
#   or
#   Hitler killed
#   Hitler elected Chancellor
def check_win(lib_policy_count, fas_policy_count, lib_win, fas_win, l_count, f_count):
  if lib_win or lib_policy_count >= 5:
    print('Liberals Win!!!')
    l_count += 1
    return True, l_count, f_count
  if fas_win or fas_policy_count >= 6:
    print('Fascists Win!!!')
    f_count += 1
    return True, l_count, f_count
  return False, l_count, f_count

# Returns Chancellor nomination vote results
def nomination_vote(president, chancellor, players, fas_policy_count):
  pass_vote = 0
  fail_vote = 0

  for player in players:
    if player.vote(president, chancellor, fas_policy_count):
      pass_vote += 1
    else:
      fail_vote += 1
  print('-----Voting-----')
  print('Pass Vote: %i' % pass_vote)
  print('Fail Vote: %i' % fail_vote)

  if pass_vote > fail_vote:
    print('Nomination Passed')
    print()
    return True
  print('Nomination Failed')
  print()
  return False

# Returns new president and index
# Special Election:
#   Occurs after 3rd Fascist policy passes
#   Current President nominates new President
def new_president(players, president_index, special_election):
  # President gets to nominate the next president
  # TODO: make it smart
  # Does a random assigment at the moment
  if special_election:
    print('-----Special Election-----')
    curr_president = players[president_index]
    # if curr_president.role == Role.Fascist:
    #   # pick another fascist
    president_index = random.randint(0, len(players) - 1)
    new_president = players[president_index]
    print('New President: ' + new_president.get_info())
    print()
    return players[president_index], president_index
  else:
    president_index = (president_index + 1) % len(players)
    return players[president_index], president_index

# Reporting
def print_game_state(lib_policy_count, fas_policy_count, failed_nom_count):
  print('-----Game State-----')
  lib = ''
  fas = ''
  for i in range(5):
    if i < lib_policy_count:
      lib += 'P-'
    else:
      lib += '0-'
  for i in range(6):
    if i < fas_policy_count:
      fas += 'P-'
    else:
      fas += '0-'
  print('Liberal Policies')
  print(lib)
  print('Fascist Policies')
  print(fas)
  print('Failed nominations: %i' % failed_nom_count)
  print()

# Checks if game ends because of Hitler election
def elected_Hitler(chancellor, fas_policy_count):
  if fas_policy_count >= 3:
    if chancellor.role == Role.HITLER:
      print('-----Elected Hitler-----')
      print()
      return True
    print(chancellor.name + ' is not Hitler')
    print()
  return False

# Cycles from players and removes the assassinated one form beliefs
def remove_from_beliefs(players, person):
  # TODO: Update beliefs
  for player in players:
    player.remove_from_beliefs(person)
  return players

# Reporting
def print_history(history):
  count = 1
  print('-----History-----')
  for turn in history:
    print("Round %i" % count)
    print("President: " + turn.president.get_info())
    print("Chancellor: " + turn.chancellor.get_info())
    if turn.passed:
      print("Vote Passed")
    else:
      print("Vote Failed")
    if turn.policy is not None:
      if turn.policy == Policy.FASCIST:
        print("Fascist Policy Passed")
      else:
        print("Liberal Policy Passed")
    if turn.reveal is not None:
      print("Investigated: " + turn.reveal[0].get_info())
      if turn.reveal[1] == Policy.FASCIST:
        print("Statement: Fascist")
      else:
        print("Statement: Liberal")
    if turn.election is not None:
      print("Special Election: " + turn.election.get_info())
    if turn.killed is not None:
      print("Assassination: " + turn.killed.get_info())
    count += 1
    print()

In [0]:
def run(l_count, f_count):
  # Focus on a 10 player game
  num_players = 10
  lib_policy_count = 0
  fas_policy_count = 0
  failed_nom_count = 0
  roles = initialize_roles(num_players)
  # Policy Tile Pile
  policies = initialize_policies()
  # Policy Discard Pile
  discard = []
  players = initialize_players(roles)
  history = [] #TODO: Create historical turn object
  president = players[0]
  chancellor = None

  fas_win = False
  lib_win = False

  finished = False
  president_index = 0
  special_election = False
  assasination = False
  round_counter = 1

  while not finished:
    print('-------------------------------------')
    print('Round #%i' % round_counter)
    print_game_state(lib_policy_count, fas_policy_count, failed_nom_count)
    print('President: ' + president.get_info())
    chancellor = president.pick_chancellor(history, fas_policy_count, lib_policy_count, failed_nom_count)
    print('Nomination: ' + chancellor.get_info())
    print()

    turn = None
    
    vote = nomination_vote(president, chancellor, players, fas_policy_count)
    passed_policy = None

    if vote:
      # vote passed
      if elected_Hitler(chancellor, fas_policy_count):
        fas_win = True
      else:
        # still playing game
        if fas_policy_count >= 3:
          # TODO: Update belief that the chancellor is not Hitler
          pass

        president_hand, policies, discard = get_policies(policies, discard)
        chancellor_hand = president.select_policies(president_hand)

        passed_policy = chancellor.play_policy(chancellor_hand)
        # Update Beliefs based on policy passed
    else:
      # vote failed
      failed_nom_count += 1
      if failed_nom_count >= 3:
        passed_policy, policies, discard = get_next_policy(policies, discard)
        failed_nom_count = 0
      else:
        history.append(Turn(president, chancellor, vote))

    if passed_policy is not None:
      failed_nom_count = 0
      if passed_policy == Policy.FASCIST:
        fas_policy_count += 1
        if fas_policy_count <= 2:
          person, statement = president.investigate()
          # Update Beliefs
          history.append(Turn(president, chancellor, vote, policy=passed_policy, pres_hand = president_hand, chan_hand=chancellor_hand, reveal=(person, statement)))
        elif fas_policy_count == 3:
          special_election = True
          # Will update beliefs later on
          # add temporary history object for belief updates
          history.append(Turn(president, chancellor, vote, policy=passed_policy, pres_hand = president_hand, chan_hand=chancellor_hand))
        else:
          assasination = True
          # add temporary histoyr object for belief updates
          history.append(Turn(president, chancellor, vote, policy=passed_policy, pres_hand = president_hand, chan_hand=chancellor_hand))
      elif passed_policy == Policy.LIBERAL:
        lib_policy_count += 1
        history.append(Turn(president, chancellor, vote, policy=passed_policy, pres_hand = president_hand, chan_hand=chancellor_hand))

    if vote and chancellor.role != Role.HITLER: 
      # only update beliefs if vote passed
      for p in players:
        p.update_beliefs(history, fas_policy_count, lib_policy_count)
    if assasination:
      # remove temporary history object
      history.pop(-1)
      person = president.assassinate()
      players.remove(person)
      if person.role == Role.HITLER:
        print('-----Hitler Killed-----')
        print(person.name + ' was Hitler')
        print()
        lib_win = True
      else:
        print(person.name + ' was not Hitler')
        print()
      # Might have out of bound issues with president_index
      # remove_from_beliefs should update the players beliefs
      players = remove_from_beliefs(players, person)
      # add complete history object
      history.append(Turn(president, chancellor, vote, policy=passed_policy, pres_hand = president_hand, chan_hand=chancellor_hand, killed=person))
      assasination = False
    if special_election:
      # remove temporary history object
      history.pop(-1)
      president, president_index = new_president(players, president_index, special_election)
      # add complete history object
      history.append(Turn(president, chancellor, vote, policy=passed_policy, pres_hand = president_hand, chan_hand=chancellor_hand, election=president))
      special_election = False
    else:
      president, president_index = new_president(players, president_index, special_election)
    finished, l_count, f_count = check_win(lib_policy_count, fas_policy_count, lib_win, fas_win, l_count, f_count)
    round_counter += 1

  print_game_state(lib_policy_count, fas_policy_count, failed_nom_count)
  print_history(history)
  return l_count, f_count


In [4]:
epochs = 100
l_count = 0
f_count = 0

for _ in range(epochs):
  l_count, f_count = run(l_count, f_count)

print('Liberal Wins: %i' % l_count)
print('Fascist Wins: %i' % f_count)


-------------------------------------
Round #1
-----Game State-----
Liberal Policies
0-0-0-0-0-
Fascist Policies
0-0-0-0-0-0-
Failed nominations: 0

President: James-Liberal
Nomination: Mary-Liberal

-----Voting-----
Pass Vote: 7
Fail Vote: 3
Nomination Passed

-----Getting Policies-----
Fascist Policy
Liberal Policy
Liberal Policy

-----President Selection-----
Liberal Policy
Liberal Policy

-----Chancellor Selection-----
Liberal Policy

-------------------------------------
Round #2
-----Game State-----
Liberal Policies
P-0-0-0-0-
Fascist Policies
0-0-0-0-0-0-
Failed nominations: 0

President: Mary-Liberal
Nomination: John-Liberal

-----Voting-----
Pass Vote: 7
Fail Vote: 3
Nomination Passed

-----Getting Policies-----
Fascist Policy
Fascist Policy
Liberal Policy

-----President Selection-----
Fascist Policy
Liberal Policy

-----Chancellor Selection-----
Liberal Policy

-------------------------------------
Round #3
-----Game State-----
Liberal Policies
P-P-0-0-0-
Fascist Policies
0-